# GridLAB-D GeoData Subcommand

As of version 4.2.21, HiPAS GridLAB-D supports the handling of geographic data.  

This command checks the version of GridLAB-D and loads the python module we will use.

In [1]:
!gridlabd --version
from IPython.display import Image
import pandas, geopandas

HiPAS GridLAB-D 4.2.21-210514


The `geodata` subcommand uses the general syntax `gridlabd geodata OPTIONS DIRECTIVE [ARGUMENTS]`.  

## Creating location data

There are two basic types of geodata entities in GridLAB-D: 

1. an unordered collection of points each specified by a `latitude,longitude` tuple; and 

1. an ordered series of waypoints along a path specified by a sequence of `latitude,longitude` tuples.  

The interpretation of a the entity is left to the dataset processor, but it can often been specified using the `location` or `position` keys, respectively.  A geodata entity can be converted from one to another by simply changing the key (see Geodata Indexing below).

The `create` directive is used to create a new geodata entity.  The general syntax is `gridlabd geodata create LOCATIONS ...`.

There are two methods of introducing locations.  The first method introduces one or more `latitude,longitude` tuples directly in the command line.  For example, the following command creates a dataset with the approximate location of SLAC's main gate.

In [2]:
!gridlabd geodata create 37.415,-122.20

id,latitude,longitude
0,37.415,-122.2


Multiple locations can be introduced by adding them to the command, for example:

In [3]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.21

id,latitude,longitude
0,37.41,-122.201
1,37.42,-122.21


The second method uses an input file with locations and associated data, such as this example CSV file:

In [4]:
!head -n 4 path_example.csv

latitude,longitude,configuration,pole_height
37.415045141688054,-122.2056472090359,flat3,15.0
37.414698020593065,-122.20848749028133,sideT,15.0
37.414454093051745,-122.21044282065421,sideT,15.0


To use this file, the following command can be used:

In [5]:
!gridlabd geodata create path_example.csv | head -n 4

id,latitude,longitude,configuration,pole_height
0,37.41505,-122.20565,flat3,15.0
1,37.4147,-122.20849,sideT,15.0
2,37.41445,-122.21044,sideT,15.0


Note that if multiple locations are provided they are sequences in the order in which they are presented, including if locations are provided directly on the command line or from data files.

If no location information is given, then the geodata is read from `/dev/stdin`, e.g.,

In [6]:
!head -n 4 path_example.csv | cut -f1-2 -d, | gridlabd geodata create

id,latitude,longitude
0,37.41505,-122.20565
1,37.4147,-122.20849
2,37.41445,-122.21044


Additional fields can be added using an expanded syntax

In [7]:
!gridlabd geodata create name=obj1+latitude=37.4205+longitude=-122.2046 name=obj2+latitude=37.5205+longitude=-122.3046

id,name,latitude,longitude
0,obj1,37.4205,-122.2046
1,obj2,37.5205,-122.3046


## Some useful options

The default precision with which latitudes and longitudes are output is 5 decimals, which is approximately 1 meter resolution. You can change the precision with which latitudes and longitudes are output using the `-p` or `--precision` option, e.g.,

In [8]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -p 4

id,latitude,longitude
0,37.41,-122.2
1,37.42,-122.2
2,37.42,-122.21


The default field separator for RAW output is a space. You can change this to any string using the `--fieldsep=STRING` option, e.g.,

In [9]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -f RAW --fieldsep=:

37.41:-122.2
37.42:-122.2
37.42:-122.21


Similarly the default record separator for RAW output is a newline. You can change this to any string using the `--recordsep=STRING` option, e.g.,

In [10]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -f RAW --recordsep=' ; '

37.41 -122.2 ; 37.42 -122.2 ; 37.42 -122.21


## Configurations
There are three locations where configuration settings are maintain: (1) system, (2) user, and (3) local.  They are consulted in this order so that the system configuration overrides the default configuration, the user configuration override the system, and the local configuration overrides the user configuration.  

By default the configuration files are named `geodata.conf`.  The system configuration is stored in `$GLD_ETC/geodata/geodata.conf` folder.  The user configuration is stored in `$HOME/.gridlabd/geodata/geodata.conf` and the local configuration is stored in `$PWD/geodata.conf`. 

The default configuration file name can be changed using the `-C FILENAME` or `--configfile FILENAME` option.

You can manage the current configuration using the `config` directive, e.g., to set the local configuration parameter `name` to `value`, use the `set` option

In [11]:
!gridlabd geodata config set name local_value

WARNING [geodata]: config file '/Users/dchassin/Documents/GitHub/slacgismo/gridlabd/gldcore/geodata/docs/geodata.conf' is not found


Note that if the file in which the parameter is stored does not already exist, you will get a warning before it is created.

To get the value, use the `get` option:

In [12]:
!gridlabd geodata config get name

local_value


To show all the configuration values, use the `show` option:

In [13]:
!gridlabd geodata config show

geodata_url='http://geodata.gridlabd.us/'
output_format='CSV'
path_join='outer'
column_names='{'ID': 'id', 'UUID': 'uuid', 'LAT': 'latitude', 'LON': 'longitude', 'DIST': 'distance', 'HEAD': 'heading', 'LOC': 'location', 'POS': 'position'}'
name='local_value'


To set a user configuration, use the `user.` prefix, e.g.,

In [14]:
!gridlabd geodata config set user.name user_value
!gridlabd geodata config show

geodata_url='http://geodata.gridlabd.us/'
output_format='CSV'
path_join='outer'
column_names='{'ID': 'id', 'UUID': 'uuid', 'LAT': 'latitude', 'LON': 'longitude', 'DIST': 'distance', 'HEAD': 'heading', 'LOC': 'location', 'POS': 'position'}'
name='local_value'
user.name='user_value'


The same syntax is used for system configuration values, e.g.,

In [15]:
!gridlabd geodata config set system.name system_value
!gridlabd geodata config show

geodata_url='http://geodata.gridlabd.us/'
output_format='CSV'
path_join='outer'
column_names='{'ID': 'id', 'UUID': 'uuid', 'LAT': 'latitude', 'LON': 'longitude', 'DIST': 'distance', 'HEAD': 'heading', 'LOC': 'location', 'POS': 'position'}'
name='local_value'
user.name='user_value'
system.name='system_value'


To remove a value, use the `unset` option, e.g.,

In [16]:
!gridlabd geodata config unset name
!gridlabd geodata config show

geodata_url='http://geodata.gridlabd.us/'
output_format='CSV'
path_join='outer'
column_names='{'ID': 'id', 'UUID': 'uuid', 'LAT': 'latitude', 'LON': 'longitude', 'DIST': 'distance', 'HEAD': 'heading', 'LOC': 'location', 'POS': 'position'}'
user.name='user_value'
system.name='system_value'


To remove all the local configuration value, simply delete the `geodata.conf` file:

In [17]:
!rm geodata.conf

## Geodata formatting

The output format can be changed using the `-f` or `--format` option. The valid formats are `CSV`, `JSON`, `RAW`, and `TABLE`.  `CSV` format is the default, but can be specified as follows:

In [18]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -f CSV

id,latitude,longitude
0,37.41,-122.2
1,37.42,-122.2
2,37.42,-122.21


JSON output looks like this:

In [19]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -f JSON

{"0":{"latitude":37.41,"longitude":-122.2},"1":{"latitude":37.42,"longitude":-122.2},"2":{"latitude":37.42,"longitude":-122.21}}


JSON output can be structured using the `pandas` `to_json()` `orient` options `index`, `split`, `records`, `columns`, `values`, or `table`, e.g.,

In [20]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -f JSON --json.orient=table

{"schema":{"fields":[{"name":"id","type":"integer"},{"name":"latitude","type":"number"},{"name":"longitude","type":"number"}],"primaryKey":["id"],"pandas_version":"0.20.0"},"data":[{"id":0,"latitude":37.41,"longitude":-122.2},{"id":1,"latitude":37.42,"longitude":-122.2},{"id":2,"latitude":37.42,"longitude":-122.21}]}


Other JSON options include `data_format`, `double_precision`, `force_ascii`, and `date_unit`. See https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_json.html for details.

RAW output is generated as follows

In [21]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -f RAW

37.41 -122.2
37.42 -122.2
37.42 -122.21


TABLE output is generated for easy reading:

In [22]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -f TABLE

    latitude  longitude
id                     
0      37.41    -122.20
1      37.42    -122.20
2      37.42    -122.21


Output formats may include an ordered field list, such as

In [23]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -f CSV:longitude,latitude

id,longitude,latitude
0,-122.2,37.41
1,-122.2,37.42
2,-122.21,37.42


## Geodata indexing

The output can be indexed using one of several standards keys, or keys from the data. The default key is `id` which is the row number, as seen above. This key can be explicitly specified as follows:

In [24]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -k id

id,latitude,longitude
0,37.41,-122.2
1,37.42,-122.2
2,37.42,-122.21


The `location` key generates a geohash code:

In [25]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -k location

location,latitude,longitude,id
9q9hg629j97y,37.41,-122.2,0
9q9hgk0em9ef,37.42,-122.2,1
9q9hgh17k9e4,37.42,-122.21,2


The `position` key generates a distance index, treating the data rows as a series of waypoints along a path. When this key is used, the distance and heading also generated.

In [26]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -k position

position,latitude,longitude,id,distance,heading
0,37.41,-122.2,0,0.0,
1111,37.42,-122.2,1,1111.95,0.0
1995,37.42,-122.21,2,1995.06,270.0


Any field or set of fields may be used for indexing, e.g.,

In [ ]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -k latitude,longitude

## Path waypoints

Waypoints can be generated along a path using the `-r` or `--resolution` option.  For example, 250-meter waypoints are generated using the following syntax:

In [ ]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -r 250 -f TABLE

The `distance` and `heading` columns are added to provide the distance and heading to the waypoint from the last location. When using waypoints, the key is automatically set to `position`, which provides the distance from the first point.  The use of other keys is not supported. In addition, any waypoint added in the process is not assigned a row `id` in order to protect the original row ids. 

The index can be changed with the `-k` or `--key` options using a pipe, e.g.,

In [ ]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -r 250 | gridlabd geodata create -k location -f TABLE

## Output

By default all output is written to `/dev/stdout`.  The output can be written to a file using the `-o` or `--output` option, e.g.,

In [ ]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -o /tmp/test.csv
!cat /tmp/test.csv

Geodata can be output to GeoPandas using the `GDF` output format, e.g.,

In [ ]:
!gridlabd geodata create 37.410,-122.20 37.420,-122.20 37.420,-122.21 -o /tmp/test.gdf -f GDF
geopandas.read_file("/tmp/test.gdf")

If the geodata does not include any geometry data, one is automatically created from the latitude and longitude fields.

----
# Datasets

The `merge` directive is used to merge a dataset into an existing geodata entity.  The general syntax is `gridlabd geodata merge -D DATASET OPTIONS`

## Distance

The `distance` dataset provide great-circle distance calculations.  For example, the following calculated the distance from the first point in the series.

In [ ]:
!gridlabd geodata merge -D distance 37.410,-122.20 37.420,-122.20 37.420,-122.21

The `distance` dataset support different units, include `meters` or `m`, `kilometers` or `km`, `feet` or `ft`, `yards` or `yd`, `miles` or `mi`.  For example, the following calculates the distances in feet:

In [ ]:
!gridlabd geodata merge -D distance 37.410,-122.20 37.420,-122.20 37.420,-122.21 --units=feet

Note that the default precision with which distances are calculated in 0 decimals. Thus the distance in `km` is rounded up to `1.0`:

In [ ]:
!gridlabd geodata merge -D distance 37.410,-122.20 37.420,-122.20 37.420,-122.21 --units=km

The precision can be changed thus: 

In [ ]:
!gridlabd geodata merge -D distance 37.410,-122.20 37.420,-122.20 37.420,-122.21 --units=km --precision.distance=2

## Address

You can use the `address` dataset to perform address resolution operations, both to and from latitude and longitude.

To obtain the address at a location, use the following command

In [ ]:
!gridlabd geodata merge -D address 37.420,-122.20

To find the latitude and longitude of an address, use the command:

In [ ]:
!gridlabd geodata merge -D address --reverse "2575 Sand Hill Road, Menlo Park CA"

You can perform both operations to resolve the "official" address from an incomplete address:

In [ ]:
!gridlabd geodata merge -D address --reverse "2575 Sand Hill Road, Menlo Park CA" | gridlabd geodata merge -D address

## Elevation
Elevation data can be obtained using the `elevation` data set, e.g.,  

In [ ]:
!gridlabd geodata merge -D elevation 37.410,-122.20 37.420,-122.20 37.420,-122.21

Elevations can be calculated in other units using the `--units=UNIT` options, e.g.,

In [ ]:
!gridlabd geodata merge -D elevation 37.410,-122.20 37.420,-122.20 37.420,-122.21 --units=feet

Valid units are `m` or `meters`, and `ft` or `feet`.

The precision of the elevations can be specified as well, e.g.,

In [ ]:
!gridlabd geodata merge -D elevation 37.410,-122.20 37.420,-122.20 37.420,-122.21 --precision.elevation=2 --units=feet

## Utility

Utility data can be obtained using the `utility` dataset.  This dataset includes information from the Homeland Infrastructure Foundation-Level Data (HIFLD).  Note that this dataset uses a very large datafile that can take some time to download the first time.

The name of the utility at a location is the default data result, e.g.,

In [ ]:
!gridlabd geodata merge -D utility 37.420,-122.20 

Other data is available (see https://hifld-geoplatform.opendata.arcgis.com/datasets/electric-retail-service-territories for detail).  A list of data available can be obtained from the dataset help, e.g., using `gridlabd geodata help utility`.  

To get the utility's summer and winter peak load and number of customers and the year for which the data is obtained, use the command

In [ ]:
!gridlabd geodata merge -D utility 37.420,-122.20 --fields=WINTR_PEAK,SUMMR_PEAK,CUSTOMERS,YEAR

The `utility` dataset also contains geometry data that can be used to generate maps, e.g.,

In [ ]:
!gridlabd geodata merge -D utility 37.420,-122.20 --geometry -f PLOT -o /tmp/utility.png --plot.figsize=10,5
Image("/tmp/utility.png")

## Powerline

Powerline calculations can be performed using the `powerline` dataset.  This dataset calculates line sag, line sway, and line gallop.  To use this dataset additional data is usually required. When the needed is not available at a specific position or location, static values can be provided using the dataset options. For example, the cable type is required and a reference to a known cable type can be given on the command line, e.g.,

In [ ]:
!gridlabd geodata merge -D powerline path_example.csv --cable_type="TACSR/AC 610mm^2" -r 250 -f TABLE:linesag

Known cables types are listed in the `geodata_powerline_cabletypes.csv`, which contains the following data:

In [ ]:
pandas.read_csv("../geodata_powerline_cabletypes.csv")

When needed data is not provided or the value provided is invalid, a warning is output and the requested values are not calculated, e.g.,

In [ ]:
!gridlabd geodata merge -D powerline path_example.csv -f TABLE:linesag

In [ ]:
!gridlabd geodata merge -D powerline path_example.csv -f TABLE:linesag --cable_type="My cabletype"

---
# Getting help

You can obtain help using the `help` directive, e.g.,

In [ ]:
!gridlabd geodata help | head -n 10

In [ ]:
!gridlabd geodata help distance | head -n 10

---
# Errors, warnings, diagnostics, and debugging

The `-v` or `--verbose` option can be used to get additional output about the stages of processing, e.g., 

In [ ]:
!gridlabd geodata merge -D distance 37,-122 38,-122 -v --units=km --precision.distance=3

The `-s` or `--silent` option suppresses error messages while still returning an error condition, e.g.,

In [ ]:
!gridlabd geodata merge -D distance 37,-122 38,-122 --badoption -s || echo "Exit code $?"

The `-w` or `--warning` option suppresses warning messages, e.g.,

In [ ]:
!gridlabd geodata merge -D distance 37,-122 38,-122 -r 100 -k location -w

The `-d` or `--debug` option can be used to obtain details about why an error occurred.  For example, the following command has an invalid `lat,lon` tuple and results in a simple error message:

In [ ]:
!gridlabd geodata merge -D distance 37,-122 38,-122 --units=furlongs

Using the debug option provides a more detailed traceback of the error that can be helpful in diagnostic the problem, particularly when it originates in a dataset package. The call arguments, options, and configuration is also output after the traceback information.

In [ ]:
!gridlabd geodata merge -D distance 37,-122 38,-122 --units=furlongs -d

Note that using the debug flag also enables traceback of warnings from inside python modules, but does not traceback warnings from geodata packages.

# Configurations and Options

The current `geodata` configuration values are obtained using the `--show_config` command line option:

In [ ]:
!gridlabd geodata --show_config

This list will always include all the `system`, `user`, and `local` configuration items.

The valid `geodata` option values are obtained using a similar command for options, e.g.,

In [ ]:
!gridlabd geodata --show_options